# Anwendung und Vergleich von Machine Learning und Deep Learning Algorithmen zur Vorhersage von COVID-19 Kennzahlen

## Regression
Mit einer Regression werden mit unabhängigen Variabeln $x^{n}$ abhängige Variabeln $y^{n}$ beschrieben. Die unabhängigen Variabeln sind numerisch. 
Die lineare Regression ist die einfachste Art der Regression. Sie geht davon aus dass die Beziehung linear ist. 
Bei der Polynomregression werden die Polynome von x verwendet: ${x}$, $x^{2}$, $x^{3}$...

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor

import plotly.graph_objs as go
from ipywidgets import interact

%matplotlib inline

# Konfiguration
Auf Windows funtktioniert eine relative Pfadangabe nicht. Darum untenstehende Konfiguration des absoluten Pfades wo das Projekt abgelegt ist.


In [4]:
#Path to raw_data -> change it localy
root_path =  "C:/Users/marco/IdeaProjects/ml-corona/"

## Laden der Daten

In [6]:
# Load the data from disk
X_train = np.load(root_path +'preprocessing/X_train.npy')
X_test = np.load(root_path +'preprocessing/X_test.npy')
y_train = np.load(root_path +'preprocessing/y_train.npy')
y_test = np.load(root_path +'preprocessing/y_test.npy')

## Modelle

### Lineare Regression

In [7]:
lr = LinearRegression(normalize=True)
scores = cross_val_score(lr, X=X_train, y=y_train, cv=10, scoring='neg_mean_absolute_error')*-1
scores

'''
https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4
splits = TimeSeriesSplit(n_splits=5)

list = []
for train_index, test_index in splits.split(X_train):
    lr.fit(X_train.iloc[train_index])
    score = lr.score()
    list.add(score)

total_score = 
'''




C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model =

array([1239.99285103,  405.27592436,  568.81584353,  367.82025817,
        242.95036199, 2679.54935609, 1018.85819342,  608.07476804,
        394.48381356,  481.47619165])

### Polynomiale Regression

In [9]:
print(X_test.shape)

(199, 10)


In [12]:
rr = Ridge(alpha=0.5)

for i in [2,3,4,5,6,7,8,9,10]:
    poly = PolynomialFeatures(i)
    X = poly.fit_transform(X_train)
    scores = cross_val_score(rr, X=X, y=y_train, cv=10, scoring='neg_mean_absolute_error')*-1
    print("With polynomial features and interactions of degree {}, we get an error of {:.2f}".format(i, np.mean(scores)))

C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.12883e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.12881e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.12842e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.12842e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:156: LinAl

With polynomial features and interactions of degree 2, we get an error of 1383.88
With polynomial features and interactions of degree 3, we get an error of 10367.74


C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.36706e-29): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anac

With polynomial features and interactions of degree 4, we get an error of 7622.80


C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_mo

...

With polynomial features and interactions of degree 8, we get an error of 11882883.74


C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_mo

With polynomial features and interactions of degree 9, we get an error of 62006241.99


C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:195: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\marco\anaconda3\lib\site-packages\sklearn\linear_mo

With polynomial features and interactions of degree 10, we get an error of 12116144.87


## Training der Modelle

## Evaluation